In [2]:
!pip install bs4

Defaulting to user installation because normal site-packages is not writeable


In [3]:
!pip install requests

Defaulting to user installation because normal site-packages is not writeable


In [4]:
!pip install lxml

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [5]:
import requests
from bs4 import BeautifulSoup

/Users/admin/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [6]:
url = "https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29"
response = requests.get(url)
response.raise_for_status()

soup = BeautifulSoup(response.text, "lxml")

In [7]:
table = soup.find("table", class_="wikitable")
a=table.find_all('tr')

In [8]:
tbody = table.find("tbody")

In [9]:
gdp = []

for i, tr in enumerate(tbody.find_all("tr")):
    row_data = {}
    for j, td in enumerate(tr.find_all("td")):
        if j == 0:  # 첫 번째 td에서 국가 이름 추출
            a_tag = td.find("a")
            if a_tag:
                row_data['Country'] = a_tag.text.strip()
            else:
                row_data['Country'] = td.text.strip()
                
        if j == 1 and td.text.strip() != "—":
            row_data['GDP_USD_billion'] = round(int(td.text.strip().replace(',', '')) / 1000, 2)
            
        if j == 2 and len(row_data) == 2:
            if len(td.text.strip()) > 4:
                row_data['Year'] = td.text.strip()[-4:]
            elif len(td.text.strip()) == 4:
                row_data['Year'] = td.text.strip()
    if len(row_data) > 1:
        gdp.append(row_data)

for g in gdp:
    print(g)

{'Country': 'World', 'GDP_USD_billion': 109529.22, 'Year': '2024'}
{'Country': 'United States', 'GDP_USD_billion': 28781.08, 'Year': '2024'}
{'Country': 'China', 'GDP_USD_billion': 18532.63, 'Year': '2024'}
{'Country': 'Germany', 'GDP_USD_billion': 4591.1, 'Year': '2024'}
{'Country': 'Japan', 'GDP_USD_billion': 4110.45, 'Year': '2024'}
{'Country': 'India', 'GDP_USD_billion': 3937.01, 'Year': '2024'}
{'Country': 'United Kingdom', 'GDP_USD_billion': 3495.26, 'Year': '2024'}
{'Country': 'France', 'GDP_USD_billion': 3130.01, 'Year': '2024'}
{'Country': 'Brazil', 'GDP_USD_billion': 2331.39, 'Year': '2024'}
{'Country': 'Italy', 'GDP_USD_billion': 2328.03, 'Year': '2024'}
{'Country': 'Canada', 'GDP_USD_billion': 2242.18, 'Year': '2024'}
{'Country': 'Russia', 'GDP_USD_billion': 2056.84, 'Year': '2024'}
{'Country': 'Mexico', 'GDP_USD_billion': 2017.03, 'Year': '2024'}
{'Country': 'Australia', 'GDP_USD_billion': 1790.35, 'Year': '2024'}
{'Country': 'South Korea', 'GDP_USD_billion': 1760.95, 'Yea

In [10]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

url = "https://en.wikipedia.org/wiki/List_of_countries_by_GDP_%28nominal%29"
response = requests.get(url)
response.raise_for_status()

soup = BeautifulSoup(response.text, "html.parser")

# HTML에서 테이블을 DataFrame으로 읽어오기
table = soup.find("table", class_="wikitable")
df = pd.read_html(str(table))[0]  # 첫 번째 테이블만 가져오기

# 컬럼 이름 설정
df.columns = ['Country', 'IMF_Forecast', 'IMF_Year', 'World_Bank_Estimate', 'World_Bank_Year', 'UN_Estimate', 'UN_Year']
print(df)

# 데이터 전처리 함수 정의
def preprocess_gdp(row):
    data = {}
    if pd.notna(row['Country']):
        data['Country'] = row['Country']
        
    if pd.notna(row['IMF_Forecast']) and row['IMF_Forecast'] != "—":
        data['GDP_USD_billion'] = round(int(row['IMF_Forecast'].replace(',', '')) / 1000, 2)
        
    if pd.notna(row['IMF_Year']):
        if len(row['IMF_Year']) > 4:
            data['Year'] = row['IMF_Year'][-4:]
        elif len(row['IMF_Year']) == 4:
            data['Year'] = row['IMF_Year']

    if len(data) > 1:
        return data
    
# 데이터 전처리 적용
gdp_data = df.apply(preprocess_gdp, axis=1).tolist()

# 결과 출력
for data in gdp_data:
    if data:
        print(data)


              Country IMF_Forecast   IMF_Year World_Bank_Estimate  \
0               World    109529216       2024           100879560   
1       United States     28781083       2024            25439700   
2               China     18532633  [n 1]2024            17963171   
3             Germany      4591100       2024             4082469   
4               Japan      4110452       2024             4256410   
..                ...          ...        ...                 ...   
209  Marshall Islands          305       2024                 258   
210      Cook Islands            —          —                   —   
211             Nauru          161       2024                 151   
212        Montserrat            —          —                   —   
213            Tuvalu           66       2024                  59   

    World_Bank_Year UN_Estimate    UN_Year  
0              2022   100834796       2022  
1              2022    25744100       2022  
2         [n 3]2022    17963170  [n 

/var/folders/14/2vn0vwzj29zb97jgpbhnrr_00000gn/T/ipykernel_16064/954727461.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]  # 첫 번째 테이블만 가져오기


In [11]:
import sqlite3

In [12]:
# 데이터베이스 연결 (파일 기반 DB를 사용)
conn = sqlite3.connect('World_Economies.db')

# 커서 생성
cursor = conn.cursor()

# 테이블 생성
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Countries_by_GDP (
        id INTEGER PRIMARY KEY,
        Country TEXT,
        GDP_USD_billion TEXT,
        Year TEXT
    )
''')

# 변경 사항 저장
conn.commit()

In [13]:
# 데이터베이스 연결 (파일 기반 DB를 사용)
conn = sqlite3.connect('World_Economies.db')

# 커서 생성
cursor = conn.cursor()

# 테이블 생성
cursor.execute('''
    DELETE FROM Countries_by_GDP
''')

# 변경 사항 저장
conn.commit()

In [14]:
# 데이터 삽입
for entry in gdp:
    cursor.execute('''
        INSERT INTO Countries_by_GDP (Country, GDP_USD_billion, Year) VALUES (?, ?, ?)
    ''', (entry['Country'], entry['GDP_USD_billion'], entry['Year']))

# 변경 사항 저장
conn.commit()

In [15]:
# 데이터 확인
cursor.execute('SELECT * FROM Countries_by_GDP')
rows = cursor.fetchall()

for row in rows:
    print(row)

# 데이터베이스 연결 종료
conn.close()

(1, 'World', '109529.22', '2024')
(2, 'United States', '28781.08', '2024')
(3, 'China', '18532.63', '2024')
(4, 'Germany', '4591.1', '2024')
(5, 'Japan', '4110.45', '2024')
(6, 'India', '3937.01', '2024')
(7, 'United Kingdom', '3495.26', '2024')
(8, 'France', '3130.01', '2024')
(9, 'Brazil', '2331.39', '2024')
(10, 'Italy', '2328.03', '2024')
(11, 'Canada', '2242.18', '2024')
(12, 'Russia', '2056.84', '2024')
(13, 'Mexico', '2017.03', '2024')
(14, 'Australia', '1790.35', '2024')
(15, 'South Korea', '1760.95', '2024')
(16, 'Spain', '1647.11', '2024')
(17, 'Indonesia', '1475.69', '2024')
(18, 'Netherlands', '1142.51', '2024')
(19, 'Turkey', '1113.56', '2024')
(20, 'Saudi Arabia', '1106.02', '2024')
(21, 'Switzerland', '938.46', '2024')
(22, 'Poland', '844.62', '2024')
(23, 'Taiwan', '802.96', '2024')
(24, 'Belgium', '655.19', '2024')
(25, 'Sweden', '623.05', '2024')
(26, 'Argentina', '604.26', '2024')
(27, 'Ireland', '564.02', '2024')
(28, 'Thailand', '548.89', '2024')
(29, 'Austria', '5